# Agents Cost Optimization

In this lab you will look various techniques including to optimize running Agentic AI Applications in Production for Enterprise Applications. The key components of cost in an Agentic AI application are 1) Inference Cost largely driven by the token count (in case of on-demand models) and model size (in case custom models or fine-tuned models.) 2) Knowledge Base - cost of ingesting the data and synchronizing the changes and cost of retrieving the context from the knowledgebase 3) Tool Calls - compute cost for executing the tools and the wait times associated or cost of the API if they are external 4) Invovation Cost - the compute involved in handling the requests from the users or the batch compute or the event triggering mechanisms.

Various techniques to optimize cost are  as follows and each of the following would be illustrated in the steps below:
- Dynamic Model Selection
- Trim prompts and outputs
- Use caching
- Batch events for inference provided use case allows

# Setup

In [ ]:
# Install the strands-agents library to the runtime.
!pip install strands-agents --quiet

In [ ]:
# Setup all the imports required for the rest of the notebook
import boto3
import json
from strands import Agent, tool
from strands.models import BedrockModel
import time
from typing import List, Dict
import utils.lab5_tools as lab_5_utils
# from utils import get_param_value
import importlib
import sagemaker

In [ ]:
# Define a model id for the US Cross Region Anthropic Claude Sonnet 3.7 Model
US_ANTHROPIC_SONNET37_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [ ]:
session = boto3.Session()

sts = session.client('sts')
identity = sts.get_caller_identity()
ACCOUNT_ID = identity['Account']
REGION = boto3.Session().region_name or 'us-west-2'

print(f"Account ID: {ACCOUNT_ID}")
print(f"Region: {REGION}")

## Dynamic Model Selection

Not all uses of Large Language applications require the same model size, for example a simple FAQ-style prompt could be handled by an Anthropic Claude Haiku model could be sufficient while a highly complex open-ended question or to sight-unseen scenarios with fluency and human-like understanding most likely will need a Claude Opus Model instead. Amazon Bedrock has a feature [Prompt Routing](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-routing.html) which allows to intelligently route requests to different models within the same model family. This feature helps to optimize for both response quality and cost.

By default Amazon Bedrock provides 1) Nova Prompt Router 2) Anthropic Prompt Router 3) Meta Prompt Router. In the below cell we will explore Bedrock's Prompt Routing feature. 

In [ ]:
# Initialize the Bedrock client
bedrock_client = boto3.client("bedrock", region_name=REGION)

# Get List of Default Prompt Routers
prompt_router_response = bedrock_client.list_prompt_routers(
    type='default'
)

# Search for the Anthropic Default Prompt Router and get its ARN
# This default prompt router model uses Claude 3 Haiku, Claude 3.5 Sonnet and switches to Claude 3.5 Sonnet as a fallback model
prompt_router_response_summaries = prompt_router_response["promptRouterSummaries"]
anthropic_default_router = [item for item in prompt_router_response_summaries 
                            if item["promptRouterName"] == "Anthropic Prompt Router"]
default_anthropic_prompt_router_arn = anthropic_default_router[0]["promptRouterArn"]

In [ ]:
# Run same set of queries through both the Default Anthropic Prompt Router and Anthropic Claude Sonnet 3.7 Model
bedrock_runtime_client = boto3.client('bedrock-runtime')

# Pricing per 1M tokens
INPUT_COST = 3.00
OUTPUT_COST = 15.00

# Test with different query complexities
test_queries = [
    "How do I return an item?",
    "Where is my order?",
    "What is Amazon Prime?",
    "Explain Amazon's return policy for electronics including restocking fees, original packaging requirements, and warranty implications for opened items.",
    "I need help with a complex return situation involving multiple international orders shipped to different addresses with various payment methods and gift card combinations."
]

print("🚀 YOUR CUSTOM ROUTER COST COMPARISON\n")

# ===== ALWAYS USE SONNET =====
print("📊 ALWAYS USE CLAUDE 3.7 SONNET:")
total_cost_always_sonnet = 0

for i, query in enumerate(test_queries):
    response, model, tokens = lab_5_utils.call_sonnet_directly(bedrock_runtime_client, US_ANTHROPIC_SONNET37_MODEL_ID, query)
    cost = lab_5_utils.router_calculate_cost("sonnet", tokens)
    total_cost_always_sonnet += cost

    print(f"  Query {i+1}: Sonnet → {tokens:,} tokens → ${cost:.6f}")

print(f"  TOTAL: ${total_cost_always_sonnet:.6f}\n")

# ===== USE YOUR CUSTOM ROUTER =====
print("⚡ USE DEFAULT ANTHROPIC ROUTER:")
total_cost_with_router = 0

for i, query in enumerate(test_queries):
    response, model, tokens = lab_5_utils.call_with_your_router(bedrock_runtime_client, default_anthropic_prompt_router_arn, query)
    cost = lab_5_utils.router_calculate_cost(model, tokens)
    total_cost_with_router += cost

    model_display = "🟢 Haiku (Fast)" if model == "haiku" else "🟡 Sonnet (Smart)"
    print(f"  Query {i+1}: {model_display} → {tokens:,} tokens → ${cost:.6f}")

print(f"  TOTAL: ${total_cost_with_router:.6f}\n")

lab_5_utils.print_prompt_router_cost_savings(total_cost_always_sonnet, total_cost_with_router)

## Trimming Prompts and Outputs

Token Count both Input and Output Tokens are a key driver for cost associated with Bedrock. There are primarily System Prompts, User Prompts and Knowledge Content that contribute to the input token count. Having optimzed prompts and knowledge is not only key to have optimal costs but also is key to maintain optimal latency for users experience. In the below example you will look how an optimal user prompt results in cost savings.

In [ ]:
# Setup
bedrock = boto3.client("bedrock-runtime", region_name=REGION)

importlib.reload(lab_5_utils)

# Amazon Customer Service Examples

print("📊 PROMPT OPTIMIZATION - COST COMPARISON")
print("Using Amazon customer service examples...")
print()

# ===== TEST UNOPTIMIZED PROMPTS =====
print("❌ UNOPTIMIZED PROMPTS (Verbose & Wasteful):")
unoptimized_total = 0

for i, prompt in enumerate(lab_5_utils.unoptimized_prompts):
    input_tokens, output_tokens, response = lab_5_utils.get_response(bedrock, US_ANTHROPIC_SONNET37_MODEL_ID, prompt)
    cost = lab_5_utils.trim_prompt_calculate_cost(input_tokens, output_tokens)
    unoptimized_total += cost

    print(f"  Query {i+1}:")
    print(f"    Input: {input_tokens} tokens")
    print(f"    Output: {output_tokens} tokens") 
    print(f"    Cost: ${cost:.6f}")
    print(f"    Response: {response[:60]}...")
    print()

print(f"📊 TOTAL UNOPTIMIZED: ${unoptimized_total:.6f}")
print()

# ===== TEST OPTIMIZED PROMPTS =====
print("✅ OPTIMIZED PROMPTS (Concise & Efficient):")
optimized_total = 0

for i, prompt in enumerate(lab_5_utils.optimized_prompts):
    input_tokens, output_tokens, response = lab_5_utils.get_response(bedrock, US_ANTHROPIC_SONNET37_MODEL_ID, prompt)
    cost = lab_5_utils.trim_prompt_calculate_cost(input_tokens, output_tokens)
    optimized_total += cost

    print(f"  Query {i+1}:")
    print(f"    Input: {input_tokens} tokens")
    print(f"    Output: {output_tokens} tokens")
    print(f"    Cost: ${cost:.6f}")
    print(f"    Response: {response[:60]}...")
    print()

print(f"✅ TOTAL OPTIMIZED: ${optimized_total:.6f}")
print()

lab_5_utils.print_prompt_router_cost_savings(unoptimized_total, optimized_total)

## Use Caching

When parts or whole of the prompt is reused in such scenarios we could benefit from using Bedrock optional feature of Prompt Caching. When using Prompt Caching the model skips recomputing parts of the prompt/context thus saving on both the response time as well as on the cost from the tokens. For example, if you have a chatbot where users can upload documents and ask questions about them, it can be time consuming for the model to process the document every time. With Prompt Caching is enabled future queries containing the document don't need to reprocess it.

Below you will see examples how to apply Cache Message Checkpoints to optimze the latency and cost of invoking the LLM.

In [ ]:
import json
import boto3
importlib.reload(lab_5_utils)

bedrock_runtime = boto3.client("bedrock-runtime", region_name=REGION)

# Pricing for Claude 3.5 Sonnet
INPUT_COST = 3.00       
OUTPUT_COST = 15.00     
CACHE_READ_COST = 0.30  

print("🚀 PROMPT CACHING DEMO - Following Blog Pattern\n")

# ===== WITHOUT CACHING =====
print("📊 WITHOUT CACHING (Fresh conversation each time):")
total_cost_no_cache = 0
lab_5_utils.clear_message_history()

for i in range(3):
    # Reset conversation for each call (no caching)
    lab_5_utils.clear_message_history()

    question = f"Based on the policy, what's the return window for laptops? (Query {i+1})"
    response, usage = lab_5_utils.converse_with_context(question, bedrock_runtime, US_ANTHROPIC_SONNET37_MODEL_ID, add_context=True, cache=False)

    cost = lab_5_utils.caching_calculate_cost(usage)
    total_cost_no_cache += cost

    print(f"  Call {i+1}:")
    print(f"    Input: {usage['inputTokens']:,}, Output: {usage['outputTokens']}")
    print(f"    Cost: ${cost:.5f}")
    print(f"    Response: {response[:100]}...")
    print()

print(f"📊 TOTAL WITHOUT CACHING: ${total_cost_no_cache:.5f}\n")

# ===== WITH CACHING =====
print("⚡ WITH CACHING (Persistent conversation):")
lab_5_utils.clear_message_history()
total_cost_with_cache = 0

# First call - establish cache
print("  Call 1 (Cache Miss - Setting up cache):")
question1 = "Based on the policy, what's the return window for laptops? (Query 1)"
response, usage = lab_5_utils.converse_with_context(question1, bedrock_runtime, US_ANTHROPIC_SONNET37_MODEL_ID, add_context=True, cache=True)

cost = lab_5_utils.caching_calculate_cost(usage)
total_cost_with_cache += cost

cache_read = usage.get("cacheReadInputTokens", 0)
print(f"    Input: {usage['inputTokens']:,}, Output: {usage['outputTokens']}, CacheRead: {cache_read:,}")
print(f"    Cost: ${cost:.5f}")
print(f"    Response: {response[:100]}...")
print()

# Subsequent calls - should hit cache
for i in range(2):
    print(f"  Call {i+2} (Cache Hit):")
    question = f"What about return fees for opened laptops? (Query {i+2})"
    response, usage = lab_5_utils.converse_with_context(question, bedrock_runtime, US_ANTHROPIC_SONNET37_MODEL_ID, add_context=False, cache=False)

    cost = lab_5_utils.caching_calculate_cost(usage)
    total_cost_with_cache += cost

    cache_read = usage.get("cacheReadInputTokens", 0)
    print(f"    Input: {usage['inputTokens']:,}, Output: {usage['outputTokens']}, CacheRead: {cache_read:,}")
    print(f"    Cost: ${cost:.5f}")
    print(f"    Response: {response[:100]}...")
    print()

print(f"⚡ TOTAL WITH CACHING: ${total_cost_with_cache:.5f}\n")
lab_5_utils.print_prompt_caching_results(total_cost_with_cache, total_cost_no_cache)

## Batch events for inference provided use case allows

For use cases that require running through several line items and where response times are not critical considering Batch inferencing is optimal. Examples of such scenarios could be sentiment analysis or summarization of vast amounts of data. Amazon Bedrock supports Batch Jobs  

In [ ]:
import boto3
import json
import time
importlib.reload(lab_5_utils)

# Setup
bedrock = boto3.client("bedrock-runtime", region_name=REGION)

# Amazon customer service prompts (realistic scenarios)
customer_queries = [
    "How do I track my Amazon order?",
    "What is the return policy for books?",
    "How do I cancel my Amazon Prime membership?",
    "I received a damaged item, what should I do?",
    "Can I return an item without the original packaging?",
    "How long does it take to get a refund on my credit card?",
    "What items are not eligible for return on Amazon?",
    "How do I contact Amazon customer service by phone?",
    "Can I change my delivery address after placing an order?",
    "What is Amazon's policy on late deliveries?"
]

print("🛒 AMAZON CUSTOMER SERVICE - ON-DEMAND vs BATCH")
print(f"Processing {len(customer_queries)} customer inquiries...")
print()

# ===== ON-DEMAND PROCESSING =====
print("⚡ ON-DEMAND PROCESSING (Immediate Response):")
on_demand_total = 0

for i, query in enumerate(customer_queries):
    input_tokens, output_tokens = lab_5_utils.process_on_demand(bedrock, US_ANTHROPIC_SONNET37_MODEL_ID, query)
    cost = lab_5_utils.batch_calculate_cost(input_tokens, output_tokens, is_batch=False)
    on_demand_total += cost

    print(f"  Query {i+1}: {input_tokens}→{output_tokens} tokens = ${cost:.6f}")

print(f"  TOTAL ON-DEMAND: ${on_demand_total:.6f}")
print()

# ===== BATCH PROCESSING =====
print("📦 BATCH PROCESSING (Delayed, but Cheaper):")
batch_results = lab_5_utils.process_batch(bedrock, US_ANTHROPIC_SONNET37_MODEL_ID, customer_queries)
batch_total = 0

for i, (input_tokens, output_tokens) in enumerate(batch_results):
    cost = lab_5_utils.batch_calculate_cost(input_tokens, output_tokens, is_batch=True)
    batch_total += cost

    print(f"  Query {i+1}: {input_tokens}→{output_tokens} tokens = ${cost:.6f}")

print(f"  TOTAL BATCH: ${batch_total:.6f}")
print()

lab_5_utils.print_batch_results(on_demand_total, batch_total, customer_queries)